In [ ]:
import csv
import math

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon
from scipy.spatial.distance import cosine

from sklearn import cluster
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
%matplotlib widget

# Cargar modelos

In [ ]:
mt0_raw_cbow = Word2Vec.load("raw/t0_lower_cbow_d256w5m3e40_n22_s5.w2v")
mt1_raw_cbow = Word2Vec.load("raw/t1_lower_cbow_d256w5m3e10_n22_s5.w2v")

mt0_raw_skip = Word2Vec.load("raw/t0_lower_skip_d256w5m3e40_n22_s5.w2v")
mt1_raw_skip = Word2Vec.load("raw/t1_lower_skip_d256w5m3e10_n22_s5.w2v")

mt0_lemmas_cbow = Word2Vec.load("lemmas/t0_lower_cbow_d256w5m3e40_n22_s5.w2v")
mt1_lemmas_cbow = Word2Vec.load("lemmas/t1_lower_cbow_d256w5m3e10_n22_s5.w2v")

mt0_lemmas_skip = Word2Vec.load("lemmas/t0_lower_skip_d256w5m3e40_n22_s5.w2v")
mt1_lemmas_skip = Word2Vec.load("lemmas/t1_lower_skip_d256w5m3e10_n22_s5.w2v")

mt0_rawpos_cbow = Word2Vec.load("raw_pos/t0_lower_cbow_d256w5m3e40_n22_s5.w2v")
mt1_rawpos_cbow = Word2Vec.load("raw_pos/t1_lower_cbow_d256w5m3e10_n22_s5.w2v")

mt0_rawpos_skip = Word2Vec.load("raw_pos/t0_lower_skip_d256w5m3e40_n22_s5.w2v")
mt1_rawpos_skip = Word2Vec.load("raw_pos/t1_lower_skip_d256w5m3e10_n22_s5.w2v")

mt0_lemmaspos_cbow = Word2Vec.load("lemmas_pos/t0_lower_cbow_d256w5m3e40_n22_s5.w2v")
mt1_lemmaspos_cbow = Word2Vec.load("lemmas_pos/t1_lower_cbow_d256w5m3e10_n22_s5.w2v")

mt0_lemaspos_skip = Word2Vec.load("lemmas_pos/t0_lower_skip_d256w5m3e40_n22_s5.w2v")
mt1_lemaspos_skip = Word2Vec.load("lemmas_pos/t1_lower_skip_d256w5m3e10_n22_s5.w2v")

# Goldstandard y diccionarios

In [3]:
GOLDEN_TARGETS = {"egemonizzare":0,"lucciola":1,"campanello":0,"trasferibile":0,"brama":0,"polisportiva":0,"palmare":1,
                  "processare":0,"pilotato":1,"cappuccio":0,"pacchetto":0,"ape":1,"unico":0,"discriminatorio":0,"rampante":1,"campionato":0,"tac":1,"piovra":0}

In [4]:
import pickle
infile = open("target_t0_dict.pkl",'rb')
target_t0_dict = pickle.load(infile)
infile.close()
infile = open("target_t1_dict.pkl",'rb')
target_t1_dict = pickle.load(infile)
infile.close()

In [5]:
def get_wordlist(dic, typ):
    word_list = []
    for word in dic.keys():
        if typ == "raw":
            word_list.append({word:dic[word]["golden"]})
        if typ == "raw_pos":
            for pos in dic[word]["pos"].keys():
                word_list.append({f"{word}_{pos}":dic[word]["golden"]})
        if typ == "lemma":
            for pos in dic[word]["pos"].keys():
                for lemma in dic[word]["pos"][pos]["lemmas"].keys():
                    if not {f"{lemma}":dic[word]["golden"]} in word_list:
                        word_list.append({f"{lemma}":dic[word]["golden"]})
        if typ == "lemma_pos":
            for pos in dic[word]["pos"].keys():
                for lemma in dic[word]["pos"][pos]["lemmas"].keys():
                    if not {f"{lemma}_{pos}":dic[word]["golden"]} in word_list:
                        word_list.append({f"{lemma}_{pos}":dic[word]["golden"]})
    return word_list

# Volcar todas las similitudes de una palabra en dos modelos a un dataframe y vizualizarlos

In [6]:
def nn2df(model0, model1, target_word):
    if target_word in list(model0.wv.vocab) and target_word in list(model1.wv.vocab):
        df0 = pd.DataFrame(model0.wv.most_similar(target_word, topn = model0.corpus_total_words), columns=["nn","t0"])
        #df0["t0_rank"] = df0["t0"].rank(ascending=False)
        df0.set_index("nn", inplace=True)
    
        df1 = pd.DataFrame(model1.wv.similar_by_vector(target_word, topn = model1.corpus_total_words), columns=["nn", "t1" ])
        #df1["t1_rank"] = abs(df1["t1"]).rank(ascending=False)
        df1.set_index("nn", inplace=True)
    
        df = pd.concat([df0,df1], axis=1)
        df.dropna(thresh=2, inplace=True)
        
        df["dif_abs"] = abs(df["t0"]-df["t1"])
        
        df["rank_0"] = df["t0"].rank()
        df["rank_1"] = df["t1"].rank()
        
        #df["dif_rank"] = abs(df["rank_0"]-df["rank_1"])
        #version_cuadratica
        df["dif_rank"] = abs(df["rank_0"]-df["rank_1"])**2
        
        df.index.name = target_word
        return df
    else:
        print(f'{target_word}  No esta en los dos modelos, no se puede comparar!!!')
        return pd.DataFrame(columns=["nn","t0","t1"])

In [7]:
def histogramas_palabras(words, column, bins):
    words_dataframe = pd.DataFrame()
    words_dataframe.index.name = "palabra_vecina"
    if type(words) == list:
        for word in words:
            w = nn2df(mt0_raw_cbow, mt1_raw_cbow, word)
            words_dataframe[word] = w[column]
        words_dataframe.hist(bins = bins,sharex=True, sharey=True)
     
"""plt.close("all")

words = ["pomodoro", "palmare"]
histogramas(words, "t0", 50)
histogramas(words, "t1", 50)
histogramas(words, "dif_abs", 50)
histogramas(words, "dif_rank", 50)"""

'plt.close("all")\n\nwords = ["pomodoro", "palmare"]\nhistogramas(words, "t0", 50)\nhistogramas(words, "t1", 50)\nhistogramas(words, "dif_abs", 50)\nhistogramas(words, "dif_rank", 50)'

# Calcular diferencias de distintos tipos 

In [86]:
def promedio_k_diferencias(nn2df,k_lista, sort_by="t0"):
    diferencias = pd.DataFrame()
    for k in k_lista:
        #Nuestro (Sin union)
        #serie_promedios = nn2df.sort_values(by=sort_by,ascending=False)[["dif_abs","dif_rank"]].head(k).mean()
        #serie_promedios.name = k
        
        #Hamilton (unido)
        # Hamilton
        dfk1 = nn2df.sort_values(by="t0", ascending=False)[["dif_abs","dif_rank"]].head(k)
        dfk2 = nn2df.sort_values(by="t1", ascending=False)[["dif_abs","dif_rank"]].head(k)
        dfk = pd.concat([dfk1, dfk2], ignore_index = True)
        dfk = dfk.drop_duplicates()
        serie_promedios = dfk[["dif_abs","dif_rank"]].head(k).mean()
        serie_promedios.name = k        

        
        diferencias = diferencias.append(serie_promedios)
    diferencias.index.name = nn2df.index.name
    return diferencias      
                         
"""w1 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro").sort_values(by="t1")
w2 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "lucciola").sort_values(by="t1")
logspace = []
[logspace.append(int(round(x))) for x in np.logspace(0.1, math.log(w.shape[0],10), base=10, num=10) if int(round(x)) not in logspace]
p1=promedio_k_diferencias(w1, logspace, sort_by="t0")
p2=promedio_k_diferencias(w2, logspace, sort_by="t0")
p = pd.DataFrame()
p["pomodoro"] = p1["dif_abs"]
p["lucciola"] = p2["dif_abs"]

p2"""

'w1 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro").sort_values(by="t1")\nw2 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "lucciola").sort_values(by="t1")\nlogspace = []\n[logspace.append(int(round(x))) for x in np.logspace(0.1, math.log(w.shape[0],10), base=10, num=10) if int(round(x)) not in logspace]\np1=promedio_k_diferencias(w1, logspace, sort_by="t0")\np2=promedio_k_diferencias(w2, logspace, sort_by="t0")\np = pd.DataFrame()\np["pomodoro"] = p1["dif_abs"]\np["lucciola"] = p2["dif_abs"]\n\np2'

In [27]:
def scoseno_k_diferencias(nn2df,k_lista, sort_by="t0"):
    scosenos = pd.DataFrame()
    for k in k_lista:
        dfk = nn2df.sort_values(by=sort_by, ascending=False)[["t0","t1","rank_0","rank_1"]].head(k)
        scos_t0_t1 = cosine_similarity(dfk['t0'].values.reshape(1, -1),dfk['t1'].values.reshape(1, -1))[0][0]
        scos_r0_r1 = cosine_similarity(dfk['rank_0'].values.reshape(1, -1),dfk['rank_1'].values.reshape(1, -1))[0][0]
        scos = pd.Series({"scos_t0_t1":scos_t0_t1, "scos_r0_r1":scos_r0_r1})
        
        #diferencia_coseno
        #scos_t0_t1 = 1-cosine_similarity(dfk['t0'].values.reshape(1, -1),dfk['t1'].values.reshape(1, -1))[0][0]
        #scos_r0_r1 = 1-cosine_similarity(dfk['rank_0'].values.reshape(1, -1),dfk['rank_1'].values.reshape(1, -1))[0][0]
        #scos = pd.Series({"scos_t0_t1":scos_t0_t1, "scos_r0_r1":scos_r0_r1})
        
        # Hamilton
        """dfk1 = nn2df.sort_values(by="t0", ascending=False)[["t0","t1","rank_0","rank_1"]].head(k)
        dfk2 = nn2df.sort_values(by="t1", ascending=False)[["t0","t1","rank_0","rank_1"]].head(k)
        dfk = pd.concat([dfk1, dfk2], ignore_index = True)
        dfk = dfk.drop_duplicates()
        scos_t0_t1 = 1-cosine_similarity(dfk['t0'].values.reshape(1, -1),dfk['t1'].values.reshape(1, -1))[0][0]
        scos_r0_r1 = 1-cosine_similarity(dfk['rank_0'].values.reshape(1, -1),dfk['rank_1'].values.reshape(1, -1))[0][0]
        scos = pd.Series({"scos_t0_t1":scos_t0_t1, "scos_r0_r1":scos_r0_r1})"""
        
        scos.name = k
        scosenos = scosenos.append(scos)
    scosenos.index.name = nn2df.index.name
    return scosenos
        
                          
"""w1 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro").sort_values(by="t1")
w2 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "lucciola").sort_values(by="t1")
logspace = []
[logspace.append(int(round(x))) for x in np.logspace(0.1, math.log(w.shape[0],10), base=10, num=10) if int(round(x)) not in logspace]
p1=scoseno_k_diferencias(w1, logspace, sort_by="t0")
p2=scoseno_k_diferencias(w2, logspace, sort_by="t0")
p = pd.DataFrame()
p["pomodoro"] = p1["scos_r0_r1"]
p["lucciola"] = p2["scos_r0_r1"]
p"""

'w1 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro").sort_values(by="t1")\nw2 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "lucciola").sort_values(by="t1")\nlogspace = []\n[logspace.append(int(round(x))) for x in np.logspace(0.1, math.log(w.shape[0],10), base=10, num=10) if int(round(x)) not in logspace]\np1=scoseno_k_diferencias(w1, logspace, sort_by="t0")\np2=scoseno_k_diferencias(w2, logspace, sort_by="t0")\np = pd.DataFrame()\np["pomodoro"] = p1["scos_r0_r1"]\np["lucciola"] = p2["scos_r0_r1"]\np'

In [17]:
def wilcoxon_k_diferencias(nn2df,k_lista, sort_by="t0"):
    wilcoxondf = pd.DataFrame()
    for k in k_lista:
        if k >= 10:
            dfk = nn2df.sort_values(by=sort_by, ascending=False)[["t0","t1","rank_0","rank_1"]].head(k)
            alpha = 0.05
            wilcox, p = wilcoxon(dfk["t0"],dfk["t1"])
            wilcox_rank, p_rank = wilcoxon(dfk["rank_0"],dfk["rank_1"])
            if p > alpha:
                '''Same distribution (fail to reject H0)'''
                w = 0   
            else:
                '''Different distribution (reject H0)'''
                w = 1
            if p_rank > alpha:
                '''Same distribution (fail to reject H0)'''
                w_rank = 0   
            else:
                '''Different distribution (reject H0)'''
                w_rank = 1
            w_series = pd.Series({"wilcox_t0_t1":w, "wilcox_r0_r1":w_rank})
            w_series.name = k
            wilcoxondf = wilcoxondf.append(w_series)
    wilcoxondf.index.name = nn2df.index.name
    return wilcoxondf
        
                          
"""w1 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro").sort_values(by="t1")
w2 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "lucciola").sort_values(by="t1")
logspace = []
[logspace.append(int(round(x))) for x in np.logspace(5, math.log(w.shape[0],10), base=10, num=20) if int(round(x)) not in logspace]
p1=wilcoxon_k_diferencias(w1, logspace, sort_by="dif_abs")
p2=wilcoxon_k_diferencias(w2, logspace, sort_by="dif_abs")
p = pd.DataFrame()
p["pomodoro"] = p1["wilcox_r0_r1"]
p["lucciola"] = p2["wilcox_r0_r1"]
p"""

'w1 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro").sort_values(by="t1")\nw2 = nn2df(mt0_raw_cbow, mt1_raw_cbow, "lucciola").sort_values(by="t1")\nlogspace = []\n[logspace.append(int(round(x))) for x in np.logspace(5, math.log(w.shape[0],10), base=10, num=20) if int(round(x)) not in logspace]\np1=wilcoxon_k_diferencias(w1, logspace, sort_by="dif_abs")\np2=wilcoxon_k_diferencias(w2, logspace, sort_by="dif_abs")\np = pd.DataFrame()\np["pomodoro"] = p1["wilcox_r0_r1"]\np["lucciola"] = p2["wilcox_r0_r1"]\np'

# Comparar entre palabras

In [18]:
def comparar_palabras(model1, model2, wordlist, compare, sort_by, k_lista):
    print("comparando")
    comparacion = pd.DataFrame()
    if type(wordlist[0]) == dict:
        for enum, word in enumerate(wordlist):
            print(f"Palabra: {list(word.keys())[0]}, {enum} de {len(wordlist)}", end="\t\t\t\t\t\t\r",flush=True)
            nn = nn2df(model1, model2, list(word.keys())[0])
            if compare[0] == "promedio":
                c = promedio_k_diferencias(nn, k_lista, sort_by="t0")[compare[1]]
            if compare[0] == "scos":
                c =scoseno_k_diferencias(nn,k_lista, sort_by="t0")[compare[1]]
            if compare[0] == "wilcox":
                c = wilcoxon_k_diferencias(nn,k_lista, sort_by="t0")[compare[1]]
            c["golden"] = word[list(word.keys())[0]]
            c.name = nn.index.name
            c.transpose()
            comparacion = comparacion.append(c)
            
            
    if type(wordlist[0]) == str:
        for word in wordlist:
            print(word, end="\t\t\r",flush=True)
            nn = nn2df(model1, model2, word)
            if compare[0] == "promedio":
                c = promedio_k_diferencias(nn, k_lista, sort_by="t0")[compare[1]]
            if compare[0] == "scos":
                c = scoseno_k_diferencias(nn,k_lista, sort_by="t0")[compare[1]]
            if compare[0] == "wilcox":
                c = wilcoxon_k_diferencias(nn,k_lista, sort_by="t0")[compare[1]]
            c.transpose()
            comparacion = comparacion.append(c)
    
    return comparacion
            
"""w = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro")
logspace = []
[logspace.append(int(round(x))) for x in np.logspace(0.1, math.log(w.shape[0],10), base=10, num=5) if int(round(x)) not in logspace]
comparacion_cbow_raw=comparar_palabras(mt0_raw_cbow, mt1_raw_cbow, ["pomodoro","lucciolla"], ["scos","scos_t0_t1"], "t0",logspace)
comparacion_cbow_raw = comparar_palabras(mt0_raw_cbow, mt1_raw_cbow, get_wordlist(target_t0_dict, "raw"), ["scos","scos_t0_t1"], "t0",logspace)"""

'w = nn2df(mt0_raw_cbow, mt1_raw_cbow, "pomodoro")\nlogspace = []\n[logspace.append(int(round(x))) for x in np.logspace(0.1, math.log(w.shape[0],10), base=10, num=5) if int(round(x)) not in logspace]\ncomparacion_cbow_raw=comparar_palabras(mt0_raw_cbow, mt1_raw_cbow, ["pomodoro","lucciolla"], ["scos","scos_t0_t1"], "t0",logspace)\ncomparacion_cbow_raw = comparar_palabras(mt0_raw_cbow, mt1_raw_cbow, get_wordlist(target_t0_dict, "raw"), ["scos","scos_t0_t1"], "t0",logspace)'

# Comparar entre grupos

In [19]:
def maximos_minimos(listo, words=False):
    max_min = pd.DataFrame()
    cambiaron = listo[listo["golden"]==1].drop("golden",axis=1)
    nocambiaron = listo[listo["golden"]==0].drop("golden",axis=1)
    print(cambiaron.shape, nocambiaron.shape)
    
    max_nocambio = nocambiaron.describe().loc["max"]
    max_nocambio.name = "max_nocambio"
    min_nocambio = nocambiaron.describe().loc["min"]
    min_nocambio.name = "min_nocambio"
    max_cambio = cambiaron.describe().loc["max"]
    max_cambio.name = "max_cambio"
    min_cambio = cambiaron.describe().loc["min"]
    min_cambio.name = "min_cambio"

    area_cambio = nocambiaron.describe().loc["max"] - nocambiaron.describe().loc["min"]
    area_cambio.name = "area_cambio"
    area_nocambio =  cambiaron.describe().loc["max"] - cambiaron.describe().loc["min"]
    area_nocambio.name = "area_nocambio"
    area_interseccion = nocambiaron.describe().loc["max"] - cambiaron.describe().loc["min"]
    area_interseccion.name = "area_interseccion"
    porcentage_inteseccion_cambio = area_interseccion/area_cambio
    porcentage_inteseccion_cambio.name = "por_inteseccion_cambio"
    porcentage_inteseccion_nocambio = area_interseccion/area_nocambio
    porcentage_inteseccion_nocambio.name = "por_inteseccion_nocambio"
    suma_porcentajes = porcentage_inteseccion_cambio+porcentage_inteseccion_nocambio
    suma_porcentajes.name = "suma_porcentajes"
    
    max_min = max_min.append(max_nocambio)
    max_min = max_min.append(min_nocambio)
    max_min = max_min.append(max_cambio)
    max_min = max_min.append(min_cambio)
    max_min = max_min.append(area_cambio)
    max_min = max_min.append(area_nocambio)
    max_min = max_min.append(area_interseccion)
    max_min = max_min.append(porcentage_inteseccion_cambio)
    max_min = max_min.append(porcentage_inteseccion_nocambio)
    max_min = max_min.append(suma_porcentajes)
    
    max_min = max_min.transpose()
    
    falsos_negativos = pd.DataFrame()
    for (columnName, columnData) in cambiaron.transpose().iteritems():
        falsos_serie = columnData < max_min["max_nocambio"]
        falsos_serie.name = columnName
        falsos_negativos = falsos_negativos.append(falsos_serie)
    max_min["FN"] = falsos_negativos.sum()
    
    falsos_positivos = pd.DataFrame()
    for (columnName, columnData) in nocambiaron.transpose().iteritems():
        falsos_serie = columnData > max_min["min_cambio"]
        falsos_serie.name = columnName
        falsos_positivos = falsos_positivos.append(falsos_serie)
    max_min["FP"] = falsos_positivos.sum()
    
    max_min["TP"] = cambiaron.shape[0] - max_min["FN"]
    max_min["TN"] = nocambiaron.shape[0] - max_min["FP"]
    
    max_min["acurracy"] = ( max_min["TP"] + max_min["TN"])/(cambiaron.shape[0]+nocambiaron.shape[0])
    max_min["precision"] = max_min["TP"]/(max_min["TP"]+max_min["FP"])
    max_min["recall"] = max_min["TP"]/(max_min["TP"]+max_min["FN"])
    
    if words:
        for word in words:
            x = listo.drop("golden",axis=1).loc[word]
            max_min[word] = x
    
    return max_min

#mm = maximos_minimos(comparacion_cbow_raw_abst0)
#mm

In [20]:
#comparacion_cbow_raw_abst0

In [35]:
def dibujar_por_goldstandard(df, porcentage=False):
    
    x = np.array(df.index,dtype=int)
    if porcentage == True :
        x = x/x.max()*100
    
    
    plt.close("all")

    fig = plt.figure(constrained_layout=True)
    gs = gridspec.GridSpec(2, 1, figure=fig)
    
    ax1 = fig.add_subplot(gs[0, 0])

    ax1.plot(x, df[["max_nocambio","min_nocambio"]], color='blue', alpha=0.5)
    ax1.plot(x, df[["max_cambio","min_cambio"]], color='orange', alpha=0.5)
    ax1.plot(x, np.full(len(x), 0.25), '--', color='black', alpha=1, label="Threshold = 0.25")
    
    #print("valores_quemados")
    #ax1.plot(x, df["polisportiva"], ".", color='blue', label="Polisportiva (Recurrent FP)")
    #ax1.plot(x, df["egemonizzare"], ".", color='red', label="Egemonizare (Recurrent FP)")
    
    ax1.fill_between(x, df["max_cambio"], df["min_cambio"], where=(df["max_cambio"] > df["min_cambio"]), color='orange', alpha=0.2, label='Range between max and min\n values for "changing words"')
    ax1.fill_between(x, df["max_nocambio"], df["min_nocambio"], where=(df["max_nocambio"] > df["min_nocambio"]), color='blue', alpha=0.2,  label='Range between max and min\n values for "non changing words"')
    ax1.fill_between(x, df["max_nocambio"], df["min_cambio"], where=(df["max_nocambio"] >  df["min_cambio"]), color='yellow', alpha=1,  label='Intersection: Indecision range')
    ax1.set_ylabel('cos_sim of $k$ nn in T0')
    #ax1.axvline(x=10, ymin=0, ymax=1, linestyle="--", label="k=10 (Our initial selection)")
    
   
    ax1.set_xscale("log", basex=10)
    #ax1.set_yscale("log", basey=10)
    legend_1 = ax1.legend(loc='bottom left', shadow=True, fontsize='x-small')
    
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.plot(x, df["acurracy"] , color='red', label='Accuracy')
    ax2.plot(x, df["precision"] , color='blue',  label='Precision')
    ax2.plot(x, df["recall"] , color='black', label='Recall')
    ax2.set_xscale("log", basex=10)
    ax2.set_xlabel('Top k nearest neighbors from the common vocabulary')
    ax2.set_ylabel('Performance\n with cos_dis=0.25')
    legend_2 = ax2.legend(loc='upper right', shadow=True, fontsize='x-small')

    

In [36]:
mm = maximos_minimos(comparacion_cbow_raw_cost0, ["polisportiva", "egemonizzare"])
dibujar_por_goldstandard(mm)
#comparacion_cbow_raw_abst0

(6, 91) (12, 91)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/001/usuarios/amnet04/anaconda3/envs/fullnlpai_fasttextbypip/lib/python3.7/site-packages/ipykernel_launcher.py:32: MatplotlibDeprecationWarning: Unrecognized location 'bottom left'. Falling back on 'best'; valid locations are
	best
	upper right
	upper left
	lower left
	lower right
	right
	center left
	center right
	lower center
	upper center
	center
This will raise an exception in 3.3.


In [24]:
target_t1_dict["polisportiva"]

{'count': 141,
 'golden': 0,
 'pos': {'propn': {'count': 63,
   'lemmas': {'polisportire': {'count': 1},
    'polisportiva': {'count': 61},
    'polisportivo': {'count': 1}}},
  'noun': {'count': 66,
   'lemmas': {'polisportiva': {'count': 65}, 'polisportivo': {'count': 1}}},
  'adj': {'count': 10,
   'lemmas': {'polisportivo': {'count': 5},
    'polisportiva': {'count': 4},
    'polisportire': {'count': 1}}},
  'verb': {'count': 2, 'lemmas': {'polisportiva': {'count': 2}}}}}

In [25]:
def dibujar_4(df1, porcentage=False):
    
    x = np.array(df.index,dtype=int)
    if porcentage == True :
        x = x/x.max()*100
    
    
    plt.close("all")

    fig = plt.figure(constrained_layout=True)
    gs = gridspec.GridSpec(1, 1, figure=fig)
    
    ax1 = fig.add_subplot(gs[0, 0])

    ax1.plot(x, df[["max_nocambio","min_nocambio"]], color='blue', alpha=0.5)
    ax1.plot(x, df[["max_cambio","min_cambio"]], color='orange', alpha=0.5)
    ax1.plot(x, np.full(len(x), 0.16), '--', color='black', alpha=1, label="Suggested threshold = 0.16")
    
    print("valores_quemados")
    ax1.plot(x, df["polisportiva"], ".", color='blue', label="Polisportiva (Recurrent FP)")
    ax1.plot(x, df["egemonizzare"], ".", color='red', label="Egemonizare (Recurrent FP)")
    
    ax1.fill_between(x, df["max_cambio"], df["min_cambio"], where=(df["max_cambio"] > df["min_cambio"]), color='orange', alpha=0.2, label='Range between max and min\n values for "changing words"')
    ax1.fill_between(x, df["max_nocambio"], df["min_nocambio"], where=(df["max_nocambio"] > df["min_nocambio"]), color='blue', alpha=0.2,  label='Range between max and min\n values for "non changing words"')
    ax1.fill_between(x, df["max_nocambio"], df["min_cambio"], where=(df["max_nocambio"] >  df["min_cambio"]), color='yellow', alpha=1,  label='Intersection: Indecision range')
    ax1.set_ylabel('avg.abs.diff\n of the target words')
    ax1.axvline(x=10, ymin=0, ymax=1, linestyle="--", label="k=10 (Our initial selection)")
    
   
    ax1.set_xscale("log", basex=10)
    #ax1.set_yscale("log", basey=10)
    legend_1 = ax1.legend(loc='upper right', shadow=True, fontsize='x-small')
    


In [26]:
mm = maximos_minimos(comparacion_cbow_raw_abst0, ["polisportiva", "egemonizzare"])
dibujar_raritas(mm)

NameError: name 'comparacion_cbow_raw_abst0' is not defined

In [ ]:
comparacion_cbow_raw_cost0.columns
comparacion_cbow_raw_cost0[[3,2]].sort_values(by=3)[:5]

In [ ]:
comparacion_cbow_raw_cost0[[3,2]].sort_values(by=2)[:5]

In [ ]:
pd.concat([comparacion_cbow_raw_cost0[[3,2]].sort_values(by=3)[:5],comparacion_cbow_raw_cost0[[3,2]].sort_values(by=2)[:5]])

In [ ]:
pd.concat([df1, df2])

In [119]:
def comparar_con_ham(df1, df2, porcentage=False):
    
    x = np.array(df1.index,dtype=int)
    if porcentage == True :
        x = x/x.max()*100
    
    plt.close("all")

    fig = plt.figure(constrained_layout=True)
    gs = gridspec.GridSpec(2, 2, figure=fig)
    
    ax1 = fig.add_subplot(gs[0, 0])

    ax1.plot(x, df1[["max_nocambio","min_nocambio"]], color='blue', alpha=0.5)
    ax1.plot(x, df1[["max_cambio","min_cambio"]], color='orange', alpha=0.5)
    ax1.plot(x, np.full(len(x), 0.25), '--', color='black', alpha=1, label="Suggested threshold = 0.25")
    
    #print("valores_quemados")
    #ax1.plot(x, df["polisportiva"], ".", color='blue', label="Polisportiva (Recurrent FP)")
    #ax1.plot(x, df["egemonizzare"], ".", color='red', label="Egemonizare (Recurrent FP)")
    
    ax1.fill_between(x, df1["max_cambio"], df1["min_cambio"], where=(df1["max_cambio"] > df1["min_cambio"]), color='orange', alpha=0.2, label='Range between max and min\n values for "changing words"')
    ax1.fill_between(x, df1["max_nocambio"], df1["min_nocambio"], where=(df1["max_nocambio"] > df1["min_nocambio"]), color='blue', alpha=0.2,  label='Range between max and min\n values for "non changing words"')
    ax1.fill_between(x, df1["max_nocambio"], df1["min_cambio"], where=(df1["max_nocambio"] >  df1["min_cambio"]), color='yellow', alpha=1,  label='Intersection: Indecision range')
    ax1.set_ylabel('cos_dis of the\n union of $k$ nn in T0 and T1')
    #ax1.axvline(x=10, ymin=0, ymax=1, linestyle="--", label="k=10 (Our initial selection)")
    
   
    ax1.set_xscale("log", basex=10)
    #ax1.set_yscale("log", basey=10)
    legend_1 = ax1.legend(loc='upper right', shadow=True, fontsize='x-small')
    
    ax2 = fig.add_subplot(gs[1, 0])
    ax2.plot(x, df1["acurracy"] , color='red', label='Accuracy')
    ax2.plot(x, df1["precision"] , color='blue',  label='Precision')
    ax2.plot(x, df1["recall"] , color='black', label='Recall')
    ax2.set_xscale("log", basex=10)
    ax2.set_xlabel('Top k nearest neighbors from the common vocabulary')
    ax2.set_ylabel('Performance\n with cos_dis=0.25')
    legend_2 = ax2.legend(loc='upper right', shadow=True, fontsize='x-small')
    
    ax3 = fig.add_subplot(gs[0, 1])
    ax3.plot(x, df2[["max_nocambio","min_nocambio"]], color='blue', alpha=0.5)
    ax3.plot(x, df2[["max_cambio","min_cambio"]], color='orange', alpha=0.5)
    ax3.plot(x, np.full(len(x), 0.25), '--', color='black', alpha=1, label="Suggested threshold = 0.25")
    
    #print("valores_quemados")
    #ax1.plot(x, df["polisportiva"], ".", color='blue', label="Polisportiva (Recurrent FP)")
    #ax1.plot(x, df["egemonizzare"], ".", color='red', label="Egemonizare (Recurrent FP)")
    
    ax3.fill_between(x, df2["max_cambio"], df2["min_cambio"], where=(df2["max_cambio"] > df2["min_cambio"]), color='orange', alpha=0.2, label='Range between max and min\n values for "changing words"')
    ax3.fill_between(x, df2["max_nocambio"], df2["min_nocambio"], where=(df2["max_nocambio"] > df2["min_nocambio"]), color='blue', alpha=0.2,  label='Range between max and min\n values for "non changing words"')
    ax3.fill_between(x, df2["max_nocambio"], df2["min_cambio"], where=(df2["max_nocambio"] >  df2["min_cambio"]), color='yellow', alpha=1,  label='Intersection: Indecision range')
    ax3.set_ylabel('cos_dis of the\n union of $k$ nn in T0 and T1')
    #ax1.axvline(x=10, ymin=0, ymax=1, linestyle="--", label="k=10 (Our initial selection)")
    
   
    ax3.set_xscale("log", basex=10)
    #ax1.set_yscale("log", basey=10)
    legend_3 = ax3.legend(loc='upper right', shadow=True, fontsize='x-small')
    
    ax4 = fig.add_subplot(gs[1, 0])
    ax4.plot(x, df2["acurracy"] , color='red', label='Accuracy')
    ax4.plot(x, df2["precision"] , color='blue',  label='Precision')
    ax4.plot(x, df2["recall"] , color='black', label='Recall')
    ax4.set_xscale("log", basex=10)
    ax4.set_xlabel('Top k nearest neighbors from the common vocabulary')
    ax4.set_ylabel('Performance\n with cos_dis=0.25')
    legend_4 = ax4.legend(loc='upper right', shadow=True, fontsize='x-small')

In [120]:
mm1 = maximos_minimos(comparacion_cbow_raw_abst0, ["polisportiva", "egemonizzare"])
mm2 = maximos_minimos(comparacion_cbow_raw_cost0, ["polisportiva", "egemonizzare"])
comparar_con_ham(mm1,mm2)

(6, 91) (12, 91)
(6, 91) (12, 91)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/001/usuarios/amnet04/anaconda3/envs/fullnlpai_fasttextbypip/lib/python3.7/site-packages/ipykernel_launcher.py:62: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.


In [ ]:
common_vocab = [x for if target_word in list(model0.wv.vocab) and target_word in list(model1.wv.vocab)]

In [128]:
list[mt0_raw_cbow.wv.vocab]

TypeError: 'type' object is not subscriptable